In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import os 
import numpy as np
from tensorflow import keras
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from methods_audio import data_handling
from methods_audio import data_augmentation
from methods_audio import denoising 
from methods_audio import model_performance_training

/Users/rosameliacarioni/miniconda3/envs/bach_thesis_4/lib/python3.9/site-packages/tensorflow_io-0.32.0-py3.9-macosx-11.0-arm64.egg/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/Users/rosameliacarioni/miniconda3/envs/bach_thesis_4/lib/python3.9/site-packages/tensorflow_io-0.32.0-py3.9-macosx-11.0-arm64.egg/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ["[Errno 2] The file to load file system plugin from does not exist.: '/Users/rosameliacarioni/miniconda3/envs/bach_thesis_4/lib/python3.9/site-packages/tensorflow_io-0.32.0-py3.9-macosx-11.0-arm64.egg/tensorflow_io/python/ops/libtensorflow_io_plugins.so'"]
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/Users/rosameliacarioni/miniconda3/envs/bach_thesis_4/lib/python3.9/site-packages/tensorflow_io-0.32.0-py3.9-macosx-11.0-arm64.egg/tensorflow_io/python/ops/__init__.py:104: UserWarning

# 1. Get data (file names)

In [2]:
data = data_handling.get_data()

Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



# 2. Read data (transforming file names into waves) <br>
Additionally, the mean is removed and the data is normalized. 

In [3]:
data = data.map(data_handling.read_in_data) 

# 3. Get input for model training 

In [4]:
samples, labels = data_handling.extract_samples_labels(data)

2023-05-10 15:27:04.413786: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


# 4. Split data into train and validation sets

In [ ]:
validation_set_size = 0.20
x_train, x_valid, y_train, y_valid = train_test_split(samples, labels, test_size= validation_set_size, random_state=123)

# 5. Data Augmentation (on train set only)

In [5]:
# Define type of augmentation to do. Options: signal, spectogram, both, none 
type_augmentation = 'none'

# 6. Denoising (on train and validation set)

In [6]:
# Define type of denoising to do. Options: spectral_gating, low_pass, none 
type_denoising= 'none'
differentiation = False 

# For low pass filter:
low_pass_cutoff = 500 # this value will be used for experimentation, ranging from 500-4000. 
low_pass_order = 4 # this value will be used for experimentation 

# 7. Transform data into Spectogram/Mel-spectogram 

In [7]:
# Define type of transformation to do. Options = spectrogram, mel_spectrogram, db_mel_spectrogram, mfcc, mfcc_delta
type_transformation = 'mel_spectrogram' 
# mel_spectogram: expected shape=(None, 624, 129, 1), found shape=(None, 624, 128, 1)
# mfcc: hape=(None, 624, 129, 1), found shape=(None, 13, 157)

# 8. Choose model

In [8]:
# Select which model to use: 
number_model = 2

batch_size = 8 
epoch = 5 

In [9]:
if (number_model == 1):
    learning_rate = 0.02661877777328162 # result from param optimization for Model 1 
elif (number_model == 2): 
    learning_rate = 0.01

# 9. Evaluate performance of the model by training it with K-Fold Validation

In [10]:
confusion_matrices, histories = model_performance_training.train_performance_k_fold(number_model, samples, labels, learning_rate, epoch, batch_size,type_augmentation, type_denoising, low_pass_cutoff, low_pass_order, type_transformation)


/Users/rosameliacarioni/University/Thesis/code/methods_audio/model_performance_training.py:88: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x_train = np.array(x)[train.astype(int)]
/Users/rosameliacarioni/University/Thesis/code/methods_audio/model_performance_training.py:90: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x_valid = np.array(x)[test.astype(int)]


Epoch 1/5
356/356 [==============================] - 18s 47ms/step - loss: 0.4133 - accuracy: 0.8357 - recall: 0.8277 - precision: 0.8401 - val_loss: 0.4403 - val_accuracy: 0.8115 - val_recall: 0.9014 - val_precision: 0.7637
Epoch 2/5
356/356 [==============================] - 16s 44ms/step - loss: 0.2796 - accuracy: 0.8896 - recall: 0.8764 - precision: 0.8993 - val_loss: 0.3020 - val_accuracy: 0.8931 - val_recall: 0.9014 - val_precision: 0.8864
Epoch 3/5
356/356 [==============================] - 16s 45ms/step - loss: 0.2152 - accuracy: 0.9173 - recall: 0.9061 - precision: 0.9264 - val_loss: 0.3298 - val_accuracy: 0.8833 - val_recall: 0.8451 - val_precision: 0.9146
Epoch 4/5
356/356 [==============================] - 17s 47ms/step - loss: 0.1536 - accuracy: 0.9420 - recall: 0.9273 - precision: 0.9549 - val_loss: 0.4429 - val_accuracy: 0.8312 - val_recall: 0.9408 - val_precision: 0.7714
Epoch 5/5
356/356 [==============================] - 16s 45ms/step - loss: 0.1123 - accuracy: 0.9599

KeyboardInterrupt: 